In [20]:
import permutationTest as pt 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time
import pickle

In [21]:
sampleShape = [10,50,100,150,200,250,300,400,500]
n_samples = 5
bins =  500
mean =0.2
std = 1

data_variables = (sampleShape, n_samples, bins, mean, std)

In [22]:
data_generator = lambda mean, std, size ,n_samples : [np.random.normal(mean, std, size) for _ in range(n_samples)]


In [11]:
def runExperiment(method_list, names_list, data_generator, data_variables):
    sampleShape, n_samples, bins, mean, std = data_variables
    
    method_dict = dict()
    
    for method, name in zip(method_list, names_list):
        
        method_dict[name] = dict()
        method_dict[name]["p_values"] = list()
        method_dict[name]["time"] = list()
        
        for i in range(n_samples):
            
            np.random.seed(i)
            
            for size in sampleShape:
                
                A = np.asarray(data_generator(mean, std, size ,1))
                B = np.asarray(data_generator(mean, std, size ,1))
        
                start = time.time()
                method_dict[name]["p_values"].append(method(A,B,bins)[0])
                end = time.time()
                method_dict[name]["time"].append(end - start)
            
    return method_dict
    

In [5]:
#method_list = [pt.GreenFloatCuda]
#names_list = ["RTX2060"]
#RTX2060_dict = runExperiment(method_list, names_list, data_generator, data_variables)

In [13]:
#pickle.dump(RTX2060_dict,open("/home/ekvall/parallelGreen/notebooks/data/experiment2/RTX2060_dict.pkl", "wb"))

In [14]:
!ls "experiment_data/experiment2"

RTX2060_dict.pkl  RTX2070_dict.pkl  TITANX_dict.pkl


In [16]:
RTX2060_dict = pickle.load(open("experiment_data/experiment2/RTX2060_dict.pkl", "rb"))
RTX2070_dict = pickle.load(open("experiment_data/experiment2/RTX2070_dict.pkl", "rb"))
TITANX_dict = pickle.load(open("experiment_data/experiment2/TITANX_dict.pkl", "rb"))

In [17]:
RTX2060_dict

{'RTX2060': {'p_values': [0.0017320141159150445,
   0.18059790722454183,
   0.3379455426075024,
   0.17633717783936032,
   0.18898194825724413,
   0.8370886299259832,
   0.8398739182931415,
   0.38444432823163427,
   0.684816478765337],
  'time': [0.2706897258758545,
   0.0073282718658447266,
   0.03447294235229492,
   0.09827828407287598,
   0.22852325439453125,
   0.4421679973602295,
   0.7910983562469482,
   2.282680034637451,
   4.7808918952941895]}}

In [18]:
RTX2070_dict

{'RTX2070': {'p_values': [0.0017320141159150445,
   0.18059790722454183,
   0.3379455426075024,
   0.17633717783936032,
   0.18898194825724413,
   0.8370886299259832,
   0.8398739182931415,
   0.38444432823163427,
   0.684816478765337],
  'time': [0.17877483367919922,
   0.016103744506835938,
   0.035025596618652344,
   0.07882094383239746,
   0.17618536949157715,
   0.33985161781311035,
   0.5744216442108154,
   1.4501307010650635,
   3.4015181064605713]}}

In [19]:
TITANX_dict

{'TITANX': {'p_values': [0.0017320141159150445,
   0.18059790722454183,
   0.3379455426075024,
   0.17633717783936032,
   0.18898194825724413,
   0.8370886299259832,
   0.8398739182931415,
   0.38444432823163427,
   0.684816478765337],
  'time': [0.5060014724731445,
   0.015324592590332031,
   0.03958487510681152,
   0.0922079086303711,
   0.20738720893859863,
   0.4067049026489258,
   0.738079309463501,
   1.9861230850219727,
   4.203988075256348]}}